In [30]:
import boto3

region = boto3.Session().region_name
session = boto3.session.Session()

ec2 = boto3.Session().client(service_name="ec2", region_name=region)
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [31]:
import sagemaker
import time
from time import gmtime, strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

iam = boto3.client("iam", config=config)

In [32]:
role_name = role.split("/")[-1]

print("Role name: {}".format(role_name))

Role name: LabRole


In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install --disable-pip-version-check -q PyAthena==2.1.0
! pip install descartes
! pip install geopandas
!pip install wordcloud
from pyathena import connect
import geopandas as gpd
from tqdm import tqdm  
from geopandas import GeoDataFrame, points_from_xy
from wordcloud import WordCloud

In [34]:
sess = sagemaker.Session()
bucket = '{}'.format(bucket)
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
bucket

'sagemaker-us-east-1-705927414280'

In [35]:
# Assign database name
database_name = "ads508"

In [36]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [37]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [38]:
# Create new database 'ads508'
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
create_db = pd.read_sql(statement, conn)
create_db

""


In [39]:
# Verify database creation
q = "SHOW DATABASES"
db_show = pd.read_sql(q, conn)
db_show

,database_name
0,ads508
1,default
2,dsoaws


In [40]:
# Set Athena parameters
database_name = "ads508"
model_table_name_csv = "modeling"
model_s3_path = "s3://{}/modeling_data".format(bucket)
print(model_s3_path)

s3://sagemaker-us-east-1-705927414280/modeling_data


In [41]:
df = pd.read_csv("{}/data_for_modeling.csv".format(model_s3_path))
df = df.drop(columns=['Unnamed: 0'])
df.head()

,latitude,longitude,n_guns_involved,target_class,group_Democrat,suspect_age,ohe_drug,ohe_officer,ohe_gang,ohe_accident,...,suspect_age_group_Adult,suspect_age_group_Senior,region_East South Central,region_Middle Atlantic,region_Mountain,region_New England,region_Pacific,region_South Atlantic,region_West North Central,region_West South Central
0,40.3467,-79.8559,1.0,1,1,Adult 18+,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,40.4555,-79.8970,1.0,1,1,NaN,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,33.9090,-118.3330,1.0,1,1,NaN,0,0,1,1,...,0,0,0,0,0,0,1,0,0,0
3,33.8447,-118.3070,1.0,1,1,Adult 18+,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,33.9454,-118.3990,1.0,1,1,NaN,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [42]:
# statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
#  latitude string,
#  longitude string,
#  n_guns_involved string,
#  target_class string,
#  group_Democrat string,
#  suspect_age string,
#  ohe_drug string,
#  ohe_officer string,
#  ohe_gang string,
#  ohe_accident string,
#  ohe_murder string,
#  ohe_suicide string,
#  ohe_arrest string,
#  ohe_brandishing string,
#  ohe_felon string,
#  ohe_drive string,
#  ohe_home_invasion string,
#  ohe_stolen string,
#  ohe_misc string,
#  ohe_drugs string,
#  ohe_car_jacking string,
#  ohe_defensive string,
#  ohe_robbery string,
#  ohe_family string,
#  ohe_institution string,
#  ohe_child string,
#  ohe_mass string,
#  ohe_domestic string,
#  suspect_age_group_Teen string,
#  suspect_age_group_Young_Adult string,
#  suspect_age_group_Mid-Adult string,
#  suspect_age_group_Adult string,
#  suspect_age_group_Senior string,
#  region_East_South_Central string,
#  region_Middle_Atlantic string,
#  region_Mountain string,
#  region_New_England string,
#  region_Pacific string,
#  region_South_Atlantic string,
#  region_West_North_Central string,
#  region_West_South_Central string
 
# ) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
# TBLPROPERTIES ('skip.header.line.count'='1')""".format(
#     database_name, model_table_name_csv, model_s3_path
# )

# create_table = pd.read_sql(statement, conn)
# create_table

### Splitting the train, test, and validation first so that when the balancing takes place next, the validation and test datasets are not affected.

In [43]:
from sklearn.model_selection import train_test_split

# Splitting all data into 90% train and 10% holdout
df_train, df_holdout = train_test_split(
        df,
        test_size=0.10,
    stratify=df['target_class'])

# Splitting holdout data into 50% validation and 50% test
df_validation, df_test = train_test_split(
        df_holdout,
        test_size=0.50,
        stratify=df_holdout['target_class'])

### Creating files for each subsection of the data: train, test, and output

In [44]:
# specifying the output file path
df_train_output = "../generated_data/df_train.csv"
df_test_output = "../generated_data/df_test.csv"
df_validation_output = "../generated_data/df_validation.csv"

# saving the DataFrame to a CSV file
df_train.to_csv(df_train_output, index=False)
df_test.to_csv(df_test_output, index=False)
df_validation.to_csv(df_validation_output, index=False)


### Displaying the initial count of the target class variable in the training dataset.
We find that the class of 1 - which indicates someone was either injured or killed - is the majority class.

In [45]:
# count the number of 0s and 1s in the 'outcome' column
value_counts = df_train['target_class'].value_counts()

# print the results
print(value_counts)

1    118401
0     79910
Name: target_class, dtype: int64


### Undersampling the majority target_class of '1' 

In [46]:
df_grouped_by = df_train.groupby(["target_class"])
df_balanced = df_grouped_by.apply(
    lambda x: x.sample(df_grouped_by.size().min())\
    .reset_index(drop=True)
)

In [47]:
# showing the balanced value counts of the newly created dataframe
value_counts_balanced = df_balanced['target_class'].value_counts()

# print the results
print(value_counts_balanced)

0    79910
1    79910
Name: target_class, dtype: int64
